# Introduction to Data Science - Lecture 6 - Loading Data, Dataframes
*COMP 5360/ MATH 4100, University of Utah, http://datasciencecourse.net/*

In this lecture, we will learn how to read in and write files and then finally cover pandas dataframes. 

# Pandas Series Recap

In the previous lab we've introduced pandas series, a one-dimensional data structure. You will see that dataframes behave very similar to series, so we will only quickly recap the most important aspects.

Let's start by importing pandas.

In [ ]:
import pandas as pd

Series have an index, and associated data. We can use the index to access the data. We've also learned that we can directly access indices and values through their implicit location. If we add the implicit location, a series looks like this: 


| Location (Implicit)| Index | Value | 
| - | - | - |
|0| Stones     |    1962
|1| Beatles    |    1960
|2| Zeppelin     |  1968
|3| Pink Floyd |    1965
|4| Pink Floyd |    2012

Here is the example in code:

In [ ]:
bands_founded = pd.Series([1962, 1960, 1968, 1965, 2012],
                         name="founded",
                         index=["Stones", "Beatles", "Zeppelin", "Pink Floyd", "Pink Floyd"])
bands_founded

Notice that when we access data with multiple indices, we don't get a simple data type, as in the above cases, but instead get another series back:

In [ ]:
bands_founded["Pink Floyd"]

## Indexing and slicing

Indexing and slicing works largely like in normal python, but instead of just directly using the bracket notations, it is recommended to use `iloc` for indexing by position and `loc` for indexing by index. 

Read the documentation on [loc](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.loc.html) and [iloc](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.iloc.html).

In [ ]:
# slicing by position
bands_founded.iloc[1:3]

In [ ]:
# slicing by index - note that the last element is included
bands_founded.loc["Zeppelin" : "Pink Floyd"]

Series can be indexed with arrays, which isn't possible in vanilla python:

In [ ]:
bands_founded.loc[["Beatles", "Pink Floyd"]]

## Masking Series

With pandas we can create boolean arrays that we can use to mask and filter a dataset. In the following expression, we'll create a new array that has "True" for every band formed after 1964 using a technique called **broadcasting**:

In [ ]:
mask = bands_founded > 1964
mask

We can use a boolean mask to filter a series:

In [ ]:
# applying the mask to the original array
bands_founded[mask]

The short form here would be:

In [ ]:
bands_founded[bands_founded > 1965]

## Exploring a Series

There are various ways to explore the data in a series. We can get an overview of the statistical properties, or calculate individual properties:

In [ ]:
numbers = pd.Series([1962, 1960, 1968, 1965, 2012, 2016])
numbers.mean()

In [ ]:
numbers.describe()

## Sorting

In [ ]:
sorted_numbers = numbers.sort_values(ascending=False)
sorted_numbers

## Resetting the Index

In [ ]:
sorted_numbers.reset_index(drop=True)

`drop` is a optional parameter with default value of false. If not passed as `True` it adds a new column with resetted index. Old index is added as a column. **This turns the series into a dataframe**

In [ ]:
sorted_numbers.reset_index() # equivalent to sorted_numbers.reset_index(drop=False)

## Mapping - Applying a Function

Mapping applies a function to every element in a series, which is a very powerful approach: 

In [ ]:
import datetime

# Convert an integer year into a date, assuming Jan 1 as day and month.
def to_date(year):
    return datetime.date(int(year), 1, 1)
    
sorted_numbers.map(to_date)

# 1. Reading Data

Up to now, we've mainly used data that we've specified directly in code. This is, of course, not particularly scalable. We want to load data from files and eventually also connect to databases and APIs. 

Data is often stored in structured file formats, such as CSV, JSON, or XML. We'll encounter all of these file formats in this class.

JSON

```json
{
    "fruit": "Apple",
    "size": "Large",
    "color": "Red"
}
```

XML
```xml
<note>
<to>Students</to>
<from>Prof</from>
<heading>Reminder</heading>
<body>HW2 due this Friday at 11:59pm!</body>
</note>
```

The simplest (and least structured) is a CSV - comma separated values - file. CSV isn't a formal file format, rather it's a table represented as a text file where the cells are separated by a delimiter. Commonly, the first row represents the header. A delimiter can be a tab character, a semicolon, a colon, etc. 

Many CSV files also have a special convention for dealing with text that could include the delimiter. The following text would be very hard to parse otherwise:
```
Artist, Album, Genre
Michael Jackson, Bad, Pop, funk, rock
``` 

Here, the album is of multiple genres which are separated by a comma. The comma, however, is also used to delimit the individual columns. To work around that, double-quotes are commonly used to indicate that all the elements contained within the quotes are not meant to be delimiters:

```
Artist, Album, Genre
Michael Jackson, Bad, "Pop, funk, rock"
``` 

Now, it is clear that `Pop, funk, rock` should belong in a single cell. 

We've prepared a dataset based on Wikipedia's [list of best-selling albums](https://en.wikipedia.org/wiki/List_of_best-selling_albums) in the file [hit_albums.csv](./hit_albums.csv). 

Here is what the first couple of lines look like:

```
Artist,Album,Released,Genre,"Certified sales (millions)",Claimed sales (millions)
Michael Jackson,Thriller,1982,"Pop, rock, R&B",45.4,65
AC/DC,Back in Black,1980,Hard rock,25.9,50
Pink Floyd,The Dark Side of the Moon,1973,Progressive rock,22.7,45
Whitney Houston / Various artists,The Bodyguard,1992,"Soundtrack/R&B, soul, pop",27.4,44
...
```

There are various ways of reading a CSV file. We'll first cover the basic read (and write) operations of Python, but will quickly move on to specific parsers for CSV files in Python and in pandas. 

## Basic File Operations

To read a file we first have to open it by specifying the file path, and specifying whether we want to read (r), write (w), both (r+), or append (a). 

In [ ]:
albums_file = open('hit_albums.csv', 'r')

We can read a whole file at once. Notice that lines are terminated with a special character, a linefeed or newline character specified as `\n`.

In [ ]:
content = albums_file.read()
content

If we print this instead, `\n` is translated into a newline: 

In [ ]:
print(content)

After reading a file, we have to manually close it again to release the OS resources:

In [ ]:
albums_file.close()

Alternatively, we can read each line separately:

In [ ]:
albums_file = open('hit_albums.csv', 'r')
line1 = albums_file.readline();
print(line1)

We could now [`split()`](https://docs.python.org/3/library/stdtypes.html#str.split) the string based on the comma, to create a simple CSV parser:

In [ ]:
line1.split(",")

We can loop over the file and read the data into an array:

In [ ]:
data = []
for line in albums_file:
    data.append(line.split(","))
    
# let's not forget to close the file:
albums_file.close()
data

We can now read individual cells or rows:

In [ ]:
data[0]

In [ ]:
data[0][1]

As we can see, this didn't take proper care of our double-quote escape of "Pop, rock, R&B". Also, numbers are still treated as strings and the newline character is also appended to the last cell. 

We could certainly improve our parser to handle these issues, but fortunately, there are existing methods to parse CSV files that make this easier.

### Writing

We can write by opening a file using the `w` flag. Here we also use the [`with`](https://docs.python.org/3/reference/compound_stmts.html#the-with-statement) keyword, which also takes care of closing the file for us, even if things go wrong (see [this blog post](https://jeffknupp.com/blog/2016/03/07/python-with-context-managers/) for details):

In [ ]:
with open('my_file.txt', 'w') as new_file:
    new_file.write("Hello World\nAre you still spinning?\n")

We can now check out this file by opening [my_file.txt](my_file.txt). Notice that the file is only guaranteed to be written if you actually close it (which, here, is take care of by the context manager invoked by the with statement). 

You can find more examples on basic file operations in the [Python Documentation](https://docs.python.org/3/tutorial/inputoutput.html).

## Exercise 1: Reading and Writing Data

The file [grades.csv](grades.csv) is a file with student names and letter grades:

```
Alice; A
Bob; B
Robert; A
Richard; C
```

Read the file into an array. Add a GPA to the student's row (A=4,B=3,C=2,D=1). 

Hint: the function [strip()](https://docs.python.org/3/library/stdtypes.html#str.strip) removes trailing whitespace from a string.

Write that file into a new file `grades_gpa.csv`

In [ ]:
gpas = {"A":4, "B":3, "C":2, "D":1}

data = []
# continue here

## Reading a CSV file with the CSV Library

We can use the CSV library to help with reading the data. It takes a `delimiter` and a `quotechar`, the latter is useful for our double quotes:

In [ ]:
# import the csv library
import csv

# initialize the top-level array
data_values = []

# open the file and append rows as arrays to the data_values
with open('hit_albums.csv') as csvfile:
    # note that we can interchangably use ' and " in general
    # for the quotechar, however we use ' so that we can use " without escaping
    filereader = csv.reader(csvfile, delimiter=',', quotechar='"')
    # the row here is an array
    for row in filereader:
        print("Row: " + str(row))
        data_values.append(row)

# Store the header in a separate array
header = data_values.pop(0)
   
print()    
print(header)
print()
print(data_values)

To do computation on the numerical dimensions of this table, we need to also convert the strings to numbers. Here, the last column, `Claimed sales (millions)` doesn't have values for each row. In that case, the conversion throws a `ValueError` exception. [Exceptions](https://docs.python.org/3/reference/compound_stmts.html#try) are error states that can be raised and caught:

In [ ]:
for row in data_values: 
    row[2] = int(row[2])
    row[4] = float(row[4])
    # need to try and catch the exception because the column contains NaN values
    try:
        row[5] = float(row[5])
    except ValueError: 
        row[5] = None
    
data_values

So, here we have matrix that we could work with. In reality, we probably would want to structure the data a little differently - instead of treating each row as an array, we'd want to treat each dimension (column) as an array, as this makes the column homogeneous and it makes it easy to calculate means, etc. 

## Reading CSV with Pandas

Now, let's take a look at what it takes to read this file using pandas.

In [ ]:
hit_albums = pd.read_csv("hit_albums.csv")
hit_albums

Well, that was different! 

Pandas provides the insanely powerful ['read_csv()'](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) method - also see [this](http://pandas.pydata.org/pandas-docs/stable/io.html) for more info on all I/O operations in pandas, including writing CSV files. 

You can pass a lot of arguments to the method, such as delimiter, quote-chars, etc., but for our case the default parameters just worked. 

We've also just created our first data frame! Let's look at data frames in detail next. 

# 2. Data Frames

A data frame is a column-oriented data structure where each column is a pandas series.

Here is [a printable cheat sheet for pandas](https://drive.google.com/drive/u/0/folders/0ByIrJAE4KMTtaGhRcXkxNHhmY2M).

We've already loaded a data frame from file, but for completeness sake, let's create one in code: 

In [ ]:
bandInfo = pd.DataFrame({
        "Name":["Led Zeppelin", "The Beatles", "Rolling Stones", "Radiohead"],
        "No Members":[4, 4, 4, 5],
        "No Albums":[9, 12, 29 ,9]
    })
bandInfo


This dataframe was initialized with a dictonary of column headers as keys and column data as values.

Just as a series, a data frame has an index, which corresponds to the first column here. In this case the index was automatically generated, but as for the series, we could use explicit values for the index. 

We can access columns in a data frame, which returns a series:

In [ ]:
bandInfo["Name"]

In [ ]:
type(bandInfo["Name"])

And obviously, we can do all the things we've learned about to this column/series. 

The previous example used columns to create the data frame. We can also create a data frame from rows. This doesn't make a ton of sense in this example, but you could find the data coming out of a data source, like our CSV file, in that order. 

In [ ]:
bandInfo2 = pd.DataFrame([
        {"Name":"Led Zeppelin", "No Albums":9, "No Members":4},
        {"Name":"The Beatles", "No Albums":12, "No Members":4},
        {"Name":"Rolling Stones", "No Albums":29, "No Members":4},
        {"Name":"Radiohead", "No Albums":9, "No Members":5},
    ])
bandInfo2

While a series has only one axis, a dataframe has two, one for the rows (the index or '0' axis), one for the columns (the column or '1' axis). We can check out these axes:

In [ ]:
bandInfo.axes

We can access these axes directly:

In [ ]:
# The row axis
bandInfo.axes[0]

In [ ]:
# The columns axis
bandInfo.axes[1]

## Exploring Data Frames

You might have noticed that data frames are rendered in nice HTML tables within Jupyter Notebooks. For small data frames, just showing all the data makes sense, but for larger datasets, like our `hit_albums` dataset, plotting 70+ rows can be annoying, and for datasets with hundreds or thousands of rows it can be prohibitive. By default, a data frame only prints a limited number of elements (notice the `...` in row 30 of the output of `hit_albums` above - only the first 30 and last 30 are printed. 

When working with data, e.g., when transforming or loading a dataset, it is important to see the raw data, for example, to check if a transformation was done correctly. Often, however, it's sufficient to see a part of the data, e.g., the first couple of rows and/or the last couple of rows. We can do this with the `head()` and `tail()` function:


In [ ]:
# head shows the first 5 rows of a datset
hit_albums.head()

In [ ]:
# we can specify how much to show
hit_albums.head(8)

In [ ]:
# tail shows the last five rows in a datasaet
hit_albums.tail()

We can check out the dimensions of the data frame:

In [ ]:
hit_albums.shape

Here we learn that our dataset has 77 rows and 6 columns.

We can also get more info about the dataset using the info method, which is especially helpful to see the data types of the columns:

In [ ]:
hit_albums.info()

As for series, we can get a rough description of the numerical values of the dataset.

In [ ]:
hit_albums.describe()

We don't see any descriptions of the columns of non-numerical type. We can, however, get a summary by directly accessing a column:

In [ ]:
hit_albums["Artist"].describe()

Here we can see that Michael Jackson is the top artist in this list, with five albums. Are there other artists with multiple albums in the list? We can answer that question with the value_counts() method:

In [ ]:
hit_albums["Artist"].value_counts()

We can look at whether the numerical columns in our data frame are correlated using the [`corr()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html) method. By default, this calculates a Pearson correlation between the column, excluding NaN values. Not surprisingly, we see a rather strong correlation (0.81) between certified and claimed sales. 

In [ ]:
hit_albums.corr()

We can also do transpose a dataframe:

In [ ]:
hit_albums.T

### Slicing Data Frames

A common task is to create subsets of a dataframe. 

We can explicitly define the the columns we want by their lables:

In [ ]:
# we only need to keep the columns we care about
hit_albums = hit_albums[["Artist","Certified sales (millions)", "Claimed sales (millions)"]]
hit_albums.head()

We can also set a column to be the index:

In [ ]:
hit_albums_reindexed = hit_albums.set_index("Artist")
hit_albums_reindexed.head()

Using these access methods we can also update the order: 

In [ ]:
hit_albums[["Certified sales (millions)", "Artist"]].head()

We can retrieve rows using the `loc` indexer by name. Note that this doesn't work if we use duplicates as indexers. 

In [ ]:
hit_albums_reindexed.loc["Green Day":"Supertramp"]

This can also be combined with slicing columns: 

In [ ]:
hit_albums_reindexed.loc["Green Day":"Supertramp", ["Certified sales (millions)"]]

Here is the same thing using `iloc`, i.e., index based slicing:

In [ ]:
hit_albums_reindexed.iloc[3:5, [0]]

### Grouping Data Frames

Very often, we want to aggregate data. Given the hit-albums dataset, for example, we might want to ask how many albums each artist in that list has sold in total. We can do these aggregations using the group-by method. 



We can specify a column to group by, for example, "Artist". We can look at the groups created:

In [ ]:
grouped = hit_albums.groupby("Artist")
grouped.groups

Note that the keys map to a set of indices. For example, Michael Jackson's albums are found at indices [0, 10, 16, 65, 66].

Once we have created these groups, we can specify what to do with it. A very generic solution is the `agg()` function, which we can pass a function to do things with the data:

In [ ]:
#here we pass the sum function, which calcualtes the sum of a list, to the group
grouped.agg(sum).head()

We can see here that we've summed up all columns. However, note that NaN plus some number is still NaN. So let's work with a slice of the dataframe instead. 


Here is an aggregation with an in-line function definition where we still create the sum, but also multiply by a million. We use a [lambda expression](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) to define the function:

In [ ]:
grouped.agg(lambda rows : sum([cell * 1000000 for cell in rows])).head()

Lambda expressions are just a different way of defining a function in line, without assigning it a name. They only work for a single statement. 

Here is a simple lambda expression, which returns a function, which we assign to the variable add:

In [ ]:
add = lambda a, b : a+b

In [ ]:
add(2, 3)

Pandas also has a couple of built in functions in place of the generic agg method. For example, we can just call `sum()`:

In [ ]:
# first we create a new slice of the dataset
artists_sales = hit_albums[["Artist","Certified sales (millions)"]]
# then we group by Artist, then sum the values up
aggregated = artists_sales.groupby("Artist").sum()
aggregated.head()

Now we sort them, and have a nice result:

In [ ]:
aggregated.sort_values("Certified sales (millions)", ascending=False).head()

### Exercise 2: Data Frames

* Calculate the mean certified sales for all albums.
* Create a new dataframe that only contains albums with more than 20 million certified sales.
* Create a new dataframe based on the hit_albums dataset that only contains the artists that have at least two albums in the list.
* Create a new dataframe that contains the aggregates sum of all certified sales for each year.

## Buit-in Plotting

Dataframes have built-in plotting capabilities based on the [matplotlib](http://matplotlib.org/) library. We'll see more about plotting later - here we'll only use the buit-in capabilities of pandas. 

First, we have to import the matplotlib library, and tell Jupyter to display the images directly here:

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
# This next line tells jupyter to render the images inline
%matplotlib inline

Then we can simply call the plot attribute:

In [ ]:
hit_albums.plot()

The default is a line chart. This doesn't make much sense, since it's mixing years with sales. We're better of plotting only the two different sales figures:

In [ ]:
hit_albums[["Certified sales (millions)", "Claimed sales (millions)"]].plot()

We can also use bar-charts instead of line-charts:

In [ ]:
hit_albums[["Certified sales (millions)", "Claimed sales (millions)"]].plot(kind="bar")

A better way to compare certified and claimed sales is a scatterplot:

In [ ]:
hit_albums.plot.scatter(x="Certified sales (millions)", y="Claimed sales (millions)")

We'll do more sophisticated plotting next time! 